# Text Generation Demo on Keras: Date Generation (One-to-Many)

In this demo, we will show you how to create a text generator using Keras. This demo is inspired by Andrew Ng's deeplearning.ai course on sequence models. (Programming Assignments: Dinosaur Island - Character-Level Language Modeling, and Jazz improvisation with LSTM)    In this demo, we create a one-to-many RNN model for generating date in the following format: e.g. "2002-03-11".  

In [1]:
import csv
import numpy as np
import random
import math
import sys

from keras.callbacks import LambdaCallback
from keras.backend import argmax,one_hot,tf
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Input, Lambda
from keras.layers import SimpleRNN
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


## Generate Dataset
We generate a toy dataset using datetime library.  The target output only comes in one format (iso format). 

In [2]:
#Generating a toy dataset
import datetime
base = datetime.datetime.today()
base = datetime.date(base.year, base.month, base.day)
date_list = [base - datetime.timedelta(days=x) for x in range(0, 1500)]
data = [date.isoformat() for date in date_list] 
print(data[0])
maxlen=10 #all the seqeunces have 10 characters

2018-03-18


In [3]:
chars = list(set(''.join(data)))
data_size, vocab_size = len(data), len(chars)
print('There are %d lines and %d unique characters in your data.' % (data_size, vocab_size))
print("max length =",maxlen)
sorted_chars= sorted(chars)

There are 1500 lines and 11 unique characters in your data.
max length = 10


In [4]:
# In this demo, we will use "<S>" as a seed character to initiate the sequence
sorted_chars.insert(0,"<S>") 
print(sorted_chars)
vocab_size = len (sorted_chars)
print(vocab_size)

['<S>', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
12


Create a dictionary to map a character to an integer, and a reverse dictionary that does the opposite.

In [5]:
char_to_ix = { ch:i for i,ch in enumerate(sorted_chars) }
ix_to_char = { i:ch for i,ch in enumerate(sorted_chars) } #reverse dictionary
print(ix_to_char)

{0: '<S>', 1: '-', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}


# Preprocessing data for Keras

In [6]:
#Preparing output data for the model
Y = []
for line in data:
    temp=[]
    for char in line:
        temp.append(char_to_ix[char]) #character to index
    Y.append(temp)
pre_Y = Y    

#Preparing input data for the model
#The first element of the sequence is an initial seed
#The rest is just like Y but shifted by one-time-step 
X = []
for item in Y:
    X.append([1]+item[:-1]) #Add initial seeed and shift X by one time step
pre_X = X        
    
#Preparing data for Keras    
X= to_categorical(X,vocab_size) #one-hot
Y= to_categorical(Y,vocab_size)
X=X.reshape(data_size,maxlen ,vocab_size)
Y=Y.reshape(data_size,maxlen ,vocab_size)
Y= np.swapaxes(Y,0,1)

print(X.shape,Y.shape)

(1500, 10, 12) (10, 1500, 12)


In [7]:
for t in range(10):
    print(X[0,t,:])

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]


# The model


In [8]:
#Shared layers (Global)
n_a = 16 #number of hidden dimensions
reshapor = Reshape((1,  vocab_size)) #Reshape the size of a tensor                         
RNN_cell = SimpleRNN(n_a, return_state = True) #An RNN Cell       
output_layer = Dense( vocab_size, activation='softmax')  #softmax output layer    

In [9]:
def train_model(Tx, n_a, n_values):
    """
    Implement the model for the training phase
    
    Arguments:
    Tx -- length of the sequence 
    n_a -- the number of hidden dimensions used in our model
    n_values -- number of unique labels in the data 
    
    Returns:
    model -- a keras model instance
    """
    
    # Define the input of your model
    X = Input(shape=(Tx, n_values))
    
    # Define a0, initial hidden state for the RNN
    a0 = Input(shape=(n_a,), name='a0')
    a = a0
    
    # Create empty list to append an output from the model in each loop
    outputs = list()
    
    # Loop  through the sequence of length Tx
    for t in range(Tx):
        # Select the "t"th time step vector from X. 
        x =  Lambda(lambda x: X[:,t,:])(X)
        # Reshape x to be (1, n_values)
        x = reshapor(x) 
        # Update the hidden state of the RNN 
        a, _ = RNN_cell(x, initial_state=[a]) 
        # Pass the hidden vector to a softmax function
        out = output_layer(a)
        # Append an output list with the current output
        outputs.append(out)
        
    # Create the model instance
    model =  Model(inputs=[X,a0], outputs=outputs)    
    return model
    

In [10]:
model = train_model(Tx = maxlen , n_a = n_a, n_values = vocab_size)
opt = Adam(lr=0.001) #optimizer
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 10, 12)        0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 12)            0           input_1[0][0]                    
____________________________________________________________________________________________________
reshape_1 (Reshape)              (None, 1, 12)         0           lambda_1[0][0]                   
                                                                   lambda_2[0][0]                   
                                                                   lambda_3[0][0]                   
                                                                   lambda_4[0][0]          

### inference model

In [11]:
def inference_model(RNN_cell, output_layer, n_values = vocab_size ,n_a = n_a, Ty = maxlen):
    """
    Implement the model for inferencing/testing phase using the parameters learned from the previous steps
    
    Arguments:
    RNN_cell -- the trained "RNN_cell" from train_model(), Keras layer object
    output_layer -- the trained "output_layer" from train_model(), Keras layer object
    n_values -- mumber of unique characters
    n_a -- number of dimensions in RNN_Cell
    Ty -- number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model
    x0 = Input(shape=(1, n_values))
    
    # Define a0, initial hidden state for the decoder RNN
    a0 = Input(shape=(n_a,), name='a0')
    a = a0
    x = x0

    # Create an empty list of "outputs" to stored the predicted outputs
    outputs = list()
    
    #Loop over Ty and generate a value at each time step
    for t in range(Ty):
        
        # Perform one step of RNN_cell 
        a, _ = RNN_cell(x, initial_state=[a])
        
        # Apply Dense softmax layer to the hidden state output of the RNN_cell
        out = output_layer(a)

        # Append an output list with the current output
        outputs.append(out)
        
        # Sample the new value to pass to the next time step
        #tf.log because tf.multinomail wants unnormalized log-prob inputs
        x  = Lambda(lambda x: one_hot(tf.multinomial(tf.log(x), 1), num_classes=vocab_size))(out)
        x = Reshape((1,vocab_size))(x)
  
    #Create the model instance
    inference_model = Model(inputs=[x0,a0], outputs=outputs)
    
    return inference_model

In [12]:
inference_model = inference_model(RNN_cell, output_layer, n_values = vocab_size, n_a = n_a, Ty = maxlen)

In [13]:
x_initializer = np.zeros((1, 1, vocab_size))
a_initializer = np.zeros((1, n_a))
inference_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1, 12)         0                                            
____________________________________________________________________________________________________
a0 (InputLayer)                  (None, 16)            0                                            
____________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)         [(None, 16), (None, 1 464         input_2[0][0]                    
                                                                   a0[0][0]                         
                                                                   reshape_2[0][0]                  
                                                                   simple_rnn_1[10][0]     

In [14]:
x_initializer[0][0][char_to_ix["<S>"]]=1 #initial seed
print(np.argmax(x_initializer[0][0]))

0


In [15]:
def generate_date(inference_model, x_initializer = x_initializer, a_initializer = a_initializer):
    """
    generate a date using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference/test time
    x_initializer -- numpy array of shape (1, 1, 12), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the RNN_cell
    
    Returns:
    text -- a generated text string
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    #predict
    pred = inference_model.predict([x_initializer,a_initializer])
    #turn predictions into integers
    indices = np.argmax(pred,axis=-1)
    #intergers to text
    text = "".join([ix_to_char[r[0]] for r in indices])
    
    return text

In [16]:
#Create a call back function
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    if(epoch%10==0):
        print()
        print('----- Generating text after Epoch: %d' % epoch)
        for i in range(3):
            text = generate_date(inference_model, x_initializer, a_initializer)
            sys.stdout.write(text)
            sys.stdout.flush()
            print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Let's train the model and generate some text

In [17]:
m = data_size
a0 = np.zeros((m, n_a))
model.fit([X, a0], list(Y),verbose=0 ,epochs=100,callbacks=[print_callback])



----- Generating text after Epoch: 0
---0---20-
----0--1--
----------

----- Generating text after Epoch: 10
2004-01-12
2014-02-20
201-2016-0

----- Generating text after Epoch: 20
2015-02-11
2016-02-11
2015-00-27

----- Generating text after Epoch: 30
2010-02-12
2010201620
2010-00-11

----- Generating text after Epoch: 40
2015-02-25
22016-00-2
2015-02-25

----- Generating text after Epoch: 50
2010-02-24
2015-02-25
2015-02-25

----- Generating text after Epoch: 60
2010-10-15
2015-02-22
2015-02-13

----- Generating text after Epoch: 70
2016-04-24
2015-02-24
2016-07-18

----- Generating text after Epoch: 80
2015-02-25
2012-07-11
2015-00-16

----- Generating text after Epoch: 90
2015-00-10
20-0001-01
2015-02-05
